In [1]:
from __future__ import unicode_literals # 이게 뭘까
import requests
import pymongo
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from scrapy.http import TextResponse
import urllib.request
import urllib
import getpass
import time
import re
from google.cloud import vision
from google.cloud.vision import types
import os
import io
from os import walk
from bs4 import BeautifulSoup


# 네이버 웹툰 썸네일 크롤링

In [2]:
def crawling_naver_webtoon_code():
    df = pd.DataFrame(columns=['title', 'genre', 'content', 'age'])
    webtoon_info = []
    total_img_count = 0

    domain_url = 'https://comic.naver.com'
    main_url = 'https://comic.naver.com/webtoon/weekday.nhn'

    html = requests.get(main_url).text
    soup = BeautifulSoup(html, 'html.parser')
    webtoon_area = soup.select('#content > div.list_area.daily_all > div.col > div > ul > li > div.thumb > a')

    for each_webtoon in webtoon_area:
        webtoon_last_page = domain_url + each_webtoon['href'] + '&page={}'.format(99999)
        html = requests.get(webtoon_last_page).text
        soup = BeautifulSoup(html, 'html.parser')
        last_page_num = int(soup.select('#content > div.paginate > div > strong.page')[0].find('em').text)

        title = ', '.join([each_ele.strip() for each_ele in soup.select('.comicinfo h2')[0].text.split('\n') if each_ele != ''])
        title = ', '.join(['&'.join([each_text.strip() for each_text in each_text.split('/')]) for each_text in title.split(',')])
        genre = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.genre')[0].text
        content = soup.select('#content > div.comicinfo > div.detail > p:nth-child(2)')[0].text
        age = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.age')[0].text

        webtoon_info.append({'title': title, 'genre': genre, 'content': content, 'age': age})

        for page_num in range(1, last_page_num + 1):
            webtoon_url = domain_url + each_webtoon['href'] + '&page={}'.format(page_num)
            html = requests.get(webtoon_url).text
            soup = BeautifulSoup(html, 'html.parser')

            if not os.path.exists('./imgs/{}'.format(title)):
                os.mkdir('./imgs/{}'.format(title))

            thumbnail_area = soup.find('table', 'viewList')
            thumbnail_img_list = thumbnail_area.select('td > a > img')[1:]

            for each_img_tag in thumbnail_img_list:
                urllib.request.urlretrieve(each_img_tag['src'], './imgs/{}/{}.png'.format(title, '&'.join(each_img_tag['alt'].split('/'))))
                total_img_count += 1

    df = pd.DataFrame(webtoon_info, columns=['title', 'genre', 'content', 'age'])
    df.to_csv('webtoon_info.csv')

In [40]:
df = pd.DataFrame(columns=['title', 'genre', 'content', 'age'])
webtoon_info = []
total_img_count = 0

domain_url = 'https://comic.naver.com'
main_url = 'https://comic.naver.com/webtoon/weekday.nhn'

html = requests.get(main_url).text
soup = BeautifulSoup(html, 'html.parser')
webtoon_area = soup.select('#content > div.list_area.daily_all > div.col > div > ul > li > div.thumb > a')

for each_webtoon in webtoon_area:
    webtoon_last_page = domain_url + each_webtoon['href'] + '&page={}'.format(99999)
    html = requests.get(webtoon_last_page).text
    soup = BeautifulSoup(html, 'html.parser')
    last_page_num = int(soup.select('#content > div.paginate > div > strong.page')[0].find('em').text)

    title = ', '.join([each_ele.strip() for each_ele in soup.select('.comicinfo h2')[0].text.split('\n') if each_ele != ''])
    title = ', '.join(['&'.join([each_text.strip() for each_text in each_text.split('/')]) for each_text in title.split(',')])
    genre = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.genre')[0].text
    content = soup.select('#content > div.comicinfo > div.detail > p:nth-child(2)')[0].text
    age = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.age')[0].text

    webtoon_info.append({'title': title, 'genre': genre, 'content': content, 'age': age})

    for page_num in range(1, last_page_num + 1):
        webtoon_url = domain_url + each_webtoon['href'] + '&page={}'.format(page_num)
        html = requests.get(webtoon_url).text
        soup = BeautifulSoup(html, 'html.parser')

        if not os.path.exists('./imgs/{}'.format(title)):
            os.mkdir('./imgs/{}'.format(title))

        thumbnail_area = soup.find('table', 'viewList')
        thumbnail_img_list = thumbnail_area.select('td > a > img')[1:]

        for each_img_tag in thumbnail_img_list:
            urllib.request.urlretrieve(each_img_tag['src'], './imgs/{}/{}.png'.format(title, '&'.join(each_img_tag['alt'].split('/'))))
            total_img_count += 1

df = pd.DataFrame(webtoon_info, columns=['title', 'genre', 'content', 'age'])
df.to_csv('webtoon_info.csv')

In [41]:
total_img_count

27860

In [42]:
df

,title,genre,content,age
0,"신의 탑, SIU","스토리, 판타지",자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년을 시험하는 탑,12세 이용가
1,"윈드브레이커, 조용석","스토리, 스포츠",자유를 꿈꾸는 청춘들의 스트릿라이딩 드라마,12세 이용가
2,"인생존망, 박태준&전선욱","에피소드, 액션","학교 다닐때 그렇게 놀았으면, 졸업하고 지금은 불행해야하는거 아니야?너 때문에 망한...",15세 이용가
3,"평범한 8반, 영파카","스토리, 드라마",안 평범한 아이들이 모인 평범한 8반고품격 웰메이드 병맛을 느껴보세요,15세 이용가
4,"귀전구담, QTT","옴니버스, 스릴러",사람들은 모여 무서운 이야기라며 귀신이야기를 한다.귀신들은 모여 귀신보다 더 무섭고...,12세 이용가
...,...,...,...,...
327,"선녀야 야옹해봐!, 햄톨탱크","스토리, 로맨스",사랑 인연을 이어주기 위해 지상에 온 선녀 초미.톱스타 강수에게 냥줍을 당하다..!!,전체연령가
328,"불발소년, 곤세","에피소드, 드라마","킬러소년 민준, 소녀를 만나고 삶이 달라지다",전체연령가
329,"갓!김치, 김민우","에피소드, 개그",​1000% 신토불이 히어로 등장!,전체연령가
330,"속삭이는 e로맨스, 최경아","스토리, 로맨스",불면증에 걸린 남자와 책만 읽으면 모두를 잠들게하는 여자가 만났다.,전체연령가
